In [1]:
from chromadb.utils.embedding_functions.onnx_mini_lm_l6_v2 import ONNXMiniLM_L6_V2 # this is how you import in Chroma 0.5.0+
# from chromadb.utils.embedding_functions import ONNXMiniLM_L6_V2 # legacy import for Chroma <=0.4.24
ef = ONNXMiniLM_L6_V2(preferred_providers=["CPUExecutionProvider"])
import chromadb
import pandas as pd

In [2]:
df = pd.read_csv('data/raw/faq_w_embeddings.csv')
df.head()

,faq,ada_embedding
0,FAQ (Please read before asking questions in th...,"[-0.09391320496797562, -0.08585827052593231, -..."
1,Please only use Chrome for this project. Unfor...,"[-0.07476626336574554, -0.12030570954084396, -..."
2,UPDATE (3/05/2024)\nThe instructions have been...,"[-0.04701472446322441, -0.11009925603866577, -..."
3,Minor errors are fine as you get acclimated to...,"[0.004953151568770409, 0.01421600952744484, -0..."
4,Please focus on multi-turn conversations with ...,"[-0.02130229026079178, 0.024433419108390808, -..."


In [3]:
client = chromadb.Client()
collection = client.get_or_create_collection("faq_embeddings", embedding_function=ef)
collection.upsert(
    documents=list(df.faq.values),
    ids=list(df.index.values.astype(str))
)

In [4]:
results = collection.query(
    query_texts=["Do i get paid for reading the instructions"], # Chroma will embed this for you
    n_results=3 # how many results to return
)
print(results)

{'ids': [['36', '3', '0']], 'embeddings': None, 'documents': [["I always just edit the coding instructions and forego the output to the user since it's not usually as important to the process. If the instructions don't mention it explicitly I believe it's ok to take liberty and do what you think makes the most sense. @Jim's approach is good too, maybe I'll try that next time.", "Minor errors are fine as you get acclimated to the instructions, but we do not expect to see blatant errors that indicate that you haven't read the instructions. Some examples are: not editing when necessary, such as when the models don't print head() and info() in the first Code Generation response of the conversation; selecting planning instruction responses that contain code/code snippets; submitting conversations in the middle of unfinished turns before the models arrive at a final answer (this does not apply to conversations that stalled/broke or hit the message limit).", "FAQ (Please read before asking qu

In [5]:
results = collection.query(
    query_texts=["The onscreen timer doesnt work"], # Chroma will embed this for you
    n_results=3 # how many results to return
)
print(results)

{'ids': [['31', '0', '15']], 'embeddings': None, 'documents': [['Use your own, separate timer for tracking payment. The timer on the screen is more related to how Data Annotation assigns tasks to different users.', "FAQ (Please read before asking questions in the chat)\nPlease read the full instructions (even if you've worked on this project before). You can log time for doing so. Note that the instructions are the same for all versions of this project. The only difference is the models that are being used.", 'For any issues that cause the conversation to stall or break in any way (meaning you\'re unable to progress the conversation), please use the checkbox at the top of the ratings window and submit the conversation. If the submission crashes, and you\'re unable to submit, you can still log your time for it. (We\'re aware of a specific issue with the "Next Message" button and/or the "Finalize Edits" button becoming unresponsive in some instances after editing an error message.)']], '

In [6]:
#chroma_client = chromadb.HttpClient(host='localhost', port=8000)